In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt


In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T #transpose
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [9]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2


In [14]:
def ReLU(Z):
    return np.maximum(Z, 0)

In [13]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [16]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

In [17]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [22]:
def ReLU_deriv(Z):
    return Z > 0

In [18]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

In [20]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [23]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [24]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)


Iteration:  0
[8 6 2 ... 3 2 2] [3 8 7 ... 3 3 2]
0.10836585365853658
Iteration:  10
[0 6 2 ... 3 3 5] [3 8 7 ... 3 3 2]
0.18802439024390244
Iteration:  20
[0 6 2 ... 3 3 2] [3 8 7 ... 3 3 2]
0.3112439024390244
Iteration:  30
[0 6 2 ... 3 3 2] [3 8 7 ... 3 3 2]
0.3954390243902439
Iteration:  40
[0 6 2 ... 3 3 2] [3 8 7 ... 3 3 2]
0.4502439024390244
Iteration:  50
[0 6 2 ... 3 3 2] [3 8 7 ... 3 3 2]
0.4933658536585366
Iteration:  60
[5 6 2 ... 3 3 2] [3 8 7 ... 3 3 2]
0.5314634146341464
Iteration:  70
[5 6 7 ... 3 3 2] [3 8 7 ... 3 3 2]
0.5635853658536585
Iteration:  80
[5 6 7 ... 3 3 2] [3 8 7 ... 3 3 2]
0.5916829268292683
Iteration:  90
[5 6 7 ... 3 3 2] [3 8 7 ... 3 3 2]
0.6156585365853658
Iteration:  100
[5 6 7 ... 3 3 2] [3 8 7 ... 3 3 2]
0.636390243902439
Iteration:  110
[5 6 7 ... 3 3 2] [3 8 7 ... 3 3 2]
0.655219512195122
Iteration:  120
[5 8 7 ... 3 3 2] [3 8 7 ... 3 3 2]
0.6716829268292683
Iteration:  130
[5 8 7 ... 3 3 2] [3 8 7 ... 3 3 2]
0.6865121951219513
Iteration:  140
[

In [28]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)


[9 2 9 7 5 1 2 5 4 8 7 4 1 6 0 1 9 6 6 6 8 1 6 2 5 6 2 1 5 6 7 7 5 4 9 1 4
 0 2 4 2 7 8 0 3 7 0 8 7 1 2 7 6 2 2 7 4 3 0 9 7 2 6 6 9 4 3 4 2 6 4 7 1 1
 1 6 2 1 6 5 6 5 7 5 5 5 0 2 3 5 2 7 4 4 6 8 4 2 5 4 0 1 1 7 7 3 8 0 5 3 7
 5 5 5 8 2 0 7 6 8 6 4 2 6 0 2 9 0 4 7 5 1 7 5 3 1 0 1 6 1 5 8 8 6 2 1 3 5
 0 2 9 3 0 3 3 6 9 4 5 1 6 6 7 8 6 2 0 2 5 9 1 3 7 8 5 4 1 6 5 3 6 8 1 3 4
 7 8 3 3 8 3 8 3 2 9 1 2 6 1 5 1 5 7 4 2 8 8 8 0 2 1 7 5 1 2 1 5 4 6 1 2 0
 0 5 3 5 9 9 1 2 2 3 4 4 9 2 4 1 1 8 6 5 5 4 4 4 5 8 9 6 2 3 6 3 2 6 6 5 1
 8 0 9 0 1 1 8 9 0 6 8 8 1 5 6 3 7 6 8 7 4 8 2 7 6 7 0 1 9 7 4 9 7 1 5 7 0
 4 0 2 6 5 6 4 9 8 8 7 5 2 9 6 1 7 3 2 1 0 8 8 4 0 6 7 2 9 6 0 2 6 1 8 7 1
 2 7 5 3 0 1 7 2 2 8 7 7 2 9 9 1 9 6 3 1 5 2 5 8 4 3 5 8 6 5 0 8 3 1 6 7 6
 7 4 6 1 4 9 5 5 1 8 0 5 1 5 9 6 1 7 7 3 3 8 3 2 1 9 9 2 7 6 9 8 4 2 0 6 3
 7 1 9 9 6 3 2 5 9 9 7 2 3 0 0 6 5 3 6 2 2 7 8 1 2 9 4 3 4 7 1 9 6 1 5 4 9
 5 4 7 5 4 1 5 1 3 0 0 6 2 3 3 4 5 2 4 8 4 7 0 3 9 0 3 3 2 1 2 9 5 6 5 0 7
 8 3 7 8 8 6 8 8 5 2 1 8 

0.862

In [27]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()
